In [1]:
data_var = '2024-01-05'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10165,2024-01-05,Brasil Nbb,19:30,União Corinthians,Flamengo,6.63,1.09,159.5,1.88,1.79,12.5,2.06,1.68,AewG4sna,0.150830,0.917431,0.531915,0.558659,0.068261,0.2,0.8,NaN,NaN,258.290,127.641920,0.494181,106.970,15.799657,0.147702,180.302,111.918,137.80,92.65,0.0,0.0,0.0,0.0,1.014863,0.034681,0.143690,-5.00,-1.000,-5.630000,0.000000,0.0,0.000000,1.56,0.312,0.288462,0.000000,0.0,0.000000
10166,2024-01-05,Brasil Nbb,19:30,Unifacisa,Franca,2.56,1.47,159.5,1.85,1.83,3.5,2.00,1.72,GYwC5NWh,0.390625,0.680272,0.540541,0.546448,0.070897,0.8,0.0,NaN,NaN,162.404,88.445014,0.544599,97.544,13.967442,0.143191,111.338,107.124,123.28,108.57,0.0,0.0,0.0,0.0,0.382504,0.007686,0.106446,1.35,0.270,5.777778,0.000000,0.0,0.000000,0.30,0.060,7.833333,0.000000,0.0,0.000000
10167,2024-01-05,Brasil Nbb,20:00,Corinthians Paulista,São José,1.65,2.15,160.5,1.80,1.86,-3.5,1.96,1.75,ObsK3115,0.606061,0.465116,0.555556,0.537634,0.071177,1.0,0.2,NaN,NaN,147.928,49.286707,0.333180,144.456,42.458618,0.293921,179.038,142.062,234.00,202.24,0.0,0.0,0.0,0.0,0.186081,0.023184,0.080050,-0.75,-0.150,-4.333333,0.000000,0.0,0.000000,-2.30,-0.460,-2.500000,0.000000,0.0,0.000000
10168,2024-01-05,Eua Nba,00:00,Golden State Warriors,Denver Nuggets,2.57,1.60,233.5,1.86,2.07,3.5,2.06,1.87,Qwl9I6fG,0.389105,0.625000,0.537634,0.483092,0.014105,0.8,0.6,NaN,NaN,212.102,87.010422,0.410229,162.564,30.821130,0.189594,169.814,176.648,202.07,136.68,0.0,0.0,0.0,0.0,0.328966,0.075569,0.068372,-0.83,-0.166,-9.457831,0.664633,0.8,0.135367,2.56,0.512,1.171875,0.582938,0.6,0.017062
10169,2024-01-05,Eua Nba,21:00,Boston Celtics,Utah Jazz,1.11,8.70,238.5,1.80,2.00,-15.5,2.10,1.67,vshDHQ9M,0.900901,0.114943,0.555556,0.500000,0.015843,0.6,0.6,NaN,NaN,175.214,37.988752,0.216813,315.972,96.440264,0.305218,156.932,333.762,158.40,447.30,0.0,0.0,0.0,0.0,1.094177,0.074432,0.161303,1.48,0.296,0.371622,0.787108,1.0,0.212892,1.54,0.308,25.000000,0.331651,0.4,0.068349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10275,2024-01-05,Eua Ncaa,21:30,Quinnipiac,Rider,1.58,2.47,148.5,1.91,1.91,-5.5,2.05,1.71,YmsKqsa5,0.632911,0.404858,0.523560,0.523560,0.037770,0.4,0.8,NaN,NaN,310.662,372.487860,1.199013,304.548,215.521301,0.707676,127.884,533.668,222.42,485.10,0.0,0.0,0.0,0.0,0.310778,0.000000,0.127881,0.41,0.082,7.073171,0.000000,0.0,0.000000,1.60,0.320,4.593750,0.000000,0.0,0.000000
10276,2024-01-05,Reino Unido Troféu Da Bbl,16:45,Newcastle Eagles,Manchester Giants,1.25,3.69,175.5,1.85,1.85,-12.5,2.08,1.63,nXpAzqvQ,0.800000,0.271003,0.540541,0.540541,0.071003,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.698518,0.000000,0.171535,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10277,2024-01-05,Rússia Premier League Feminina,12:00,Nika Siktivkar F,Spartak Moscow F,1.06,7.70,137.5,1.85,1.85,-16.5,2.03,1.67,CMzzuoK5,0.943396,0.129870,0.540541,0.540541,0.073266,0.0,0.0,NaN,NaN,207.146,165.080392,0.796928,376.494,272.914618,0.724884,0.000,0.000,426.98,142.29,1.0,0.0,0.0,0.0,1.071961,0.000000,0.137599,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10278,2024-01-05,Sérvia Primeira Liga,12:00,Dynamic,OKK Beograd,1.47,2.56,167.5,1.81,1.89,-4.5,1.84,1.86,Ojy28PTD,0.680272,0.390625,0.552486,0.529101,0.0

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Golden State Warriors,Denver Nuggets,233.5,1.86,1.0000,Over
1,23:00,Eua Nba,Denver Nuggets,Orlando Magic,224.5,1.85,0.9984,Over
2,16:30,Europa Euroliga,Barcelona,Baskonia,163.5,1.85,1.0000,Over
